In [41]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


# 获取网页
def askurl(url):
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36 QIHU 360SE"
    }
    html = requests.get(url, headers=headers)
    return html.text

# 获取易车的所以吉利车型
def get_jili_brands():
    brands = []
    cars = open('./易车.html')
    brand = BeautifulSoup(cars)
    brand = brand.find('div', class_='yccmp-subbrand-wrap')
    jili_cars = brand.find_all('div')
    for jili_car in jili_cars:
        jili_a = jili_car.find_all('a')
        for jili in jili_a:
            span1 = jili.find('span', class_='yccmp-subbrand-item-tit')
            span2 = jili.find('span', class_='yccmp-subbrand-item-sale')
            if len(span2.text) != 0: continue
            href = jili['href']
            brands.append([span1.text, href])
    return brands

# 分割字符串：10.12-18.53万：[10.12, 18.53]
def split_str(str):
    site_1 = str.find('-')
    str1 = str[:site_1]
    site_2 = str.find('万')
    str2 = str[site_1+1:site_2-1]
    return str1.lstrip(), str2

# 获取吉利汽车所有竞品
def get_cars():
    all_brands = []
    brands = get_jili_brands()
    for brand in brands:
        html = askurl(brand[1])
        html = BeautifulSoup(html)
        jili_price = html.find('div', class_='block1 guidance-price')
        jili_price = jili_price.find('span').text
        # 吉利汽车的最低价格和最高价格
        low_price, high_price = split_str(jili_price)
        # 获取竞品
        other_brands = html.find('div', class_='chexingPK-box').find_all('dl')
        for other_brand in other_brands:
            href = other_brand.find('a')['href']
            href = 'https://car.yiche.com/' + href
            name = other_brand.find('p', class_='name').text
            other_low_price, other_high_price = split_str(other_brand.find('p',class_='num').text)
            new_page = askurl(href)
            new_page = BeautifulSoup(new_page)
            new_page_fl = new_page.find('div', class_='info-box').find('div', class_='fl').find_all('li')
            new_page_fr = new_page.find('div', class_='info-box').find('div', class_='fr').find_all('li')
            if new_page_fl[0].text == name: new_page_brand = new_page_fl[3].text
            else: new_page_brand = new_page_fr[3].text
            all_brands.append([brand[0], low_price, high_price, new_page_brand, name, other_low_price, other_high_price])
    all_brands = np.array(all_brands)
    all_brands = pd.DataFrame({'吉利车型': all_brands[:, 0], '低配价格': all_brands[:, 1], '高配价格': all_brands[:, 2], '其它品牌': all_brands[:, 3],'其它车型': all_brands[:, 4], '其它车型低配价格': all_brands[:, 5], '其它车型高配价格': all_brands[:, 6]})
    print(all_brands)
    all_brands.to_csv("易车.csv")

# print(askurl('https://car.autohome.com.cn/price/brand-25.html'))
# 汽车之家的三级网络获取有待商榷，可能存疑
# html = askurl('https://www.yiche.com/')
# print(html)
# get_all_brands()
get_cars()

       吉利车型   低配价格  高配价格  其它品牌     其它车型 其它车型低配价格 其它车型高配价格
0        缤越   7.58  11.9  长安欧尚   长安欧尚Z6     9.99     12.9
1        缤越   7.58  11.9    荣威    荣威RX5     9.33     13.5
2        缤越   7.58  11.9  长安欧尚   长安欧尚X5     6.99     10.9
3        缤越   7.58  11.9  广汽传祺    传祺GS4     8.98     13.1
4        缤越   7.58  11.9  五菱汽车     五菱星辰     6.98      9.9
..      ...    ...   ...   ...      ...      ...      ...
139  博瑞PHEV  15.58  16.9    大众       凌渡    15.09     19.0
140  博瑞PHEV  15.58  16.9    丰田      亚洲狮    14.18     17.9
141  博瑞PHEV  15.58  16.9    欧拉     欧拉好猫    14.10     17.1
142  博瑞PHEV  15.58  16.9    丰田       凌尚    14.88     15.8
143  博瑞PHEV  15.58  16.9   雪铁龙  凡尔赛C5 X    14.37     18.6

[144 rows x 7 columns]
